In [10]:
import os
import csv
import sqlite3

# Define input and output paths
input_path = "spider-schema.csv"
output_base_dir ="spider_sqlite"  # Change this to your desired output location

with open(input_path) as f:
    databases = {}

    csvfile = csv.reader(f, skipinitialspace=True)
    header = None
    for line in csvfile:
        if header is None:
            header = line
            continue
        row = dict(zip(header, line))
        
        db = row["Database name"].lower()
        table = row["Table Name"].lower()
        column = row["Field Name"].lower()
        column_type = row["Type"]
        column_primray = row["Is Primary Key"]
        if db not in databases:
            databases[db] = {"tables": {}}
        if table not in databases[db]["tables"]:
            databases[db]["tables"][table] = {"columns": {}, "primary": []}
        if column not in databases[db]["tables"][table]["columns"]:
            databases[db]["tables"][table]["columns"][column] = { "name": column, "type": column_type }
            if column_primray == "True":
                databases[db]["tables"][table]["primary"].append(column)

    # Create output directory if it doesn't exist
    os.makedirs(output_base_dir, exist_ok=True)

    for db in databases:
        db_dir = os.path.join(output_base_dir, db)
        os.makedirs(db_dir, exist_ok=True)
        db_path = os.path.join(db_dir, f"{db}.sqlite")
        dbconn = sqlite3.connect(db_path)
        dbcur = dbconn.cursor()

        for table in databases[db]["tables"]:
            if "sqlite_sequence" == table:
                continue
            tablesql = "CREATE TABLE " + table + "("
            coldelim = " "
            for col in databases[db]["tables"][table]["columns"]:
                col = databases[db]["tables"][table]["columns"][col]
                tablesql += coldelim + '"' + col["name"] + '" ' + col["type"]
                coldelim = ","
            if len(databases[db]["tables"][table]["primary"]):
                tablesql += ",PRIMARY KEY ("+ ",".join(databases[db]["tables"][table]["primary"]) +")"
            tablesql += ");"
            print(tablesql)
            dbcur.execute(tablesql)
        dbconn.commit()
        
    f.close()
    print(f"\nYou can now use database files in {output_base_dir}\n")

CREATE TABLE perpetrator( "perpetrator_id" number,"people_id" number,"date" text,"year" number,"location" text,"country" text,"killed" number,"injured" number,PRIMARY KEY (perpetrator_id));
CREATE TABLE people( "people_id" number,"name" text,"height" number,"weight" number,"home_town" text,PRIMARY KEY (people_id));
CREATE TABLE classroom( "building" text,"room_number" text,"capacity" number,PRIMARY KEY (building));
CREATE TABLE department( "department_name" text,"building" text,"budget" number,PRIMARY KEY (department_name));
CREATE TABLE course( "course_id" text,"title" text,"department_name" text,"credits" number,PRIMARY KEY (course_id));
CREATE TABLE instructor( "id" text,"name" text,"department_name" text,"salary" number,PRIMARY KEY (id));
CREATE TABLE section( "course_id" text,"section_id" text,"semester" text,"year" number,"building" text,"room_number" text,"time_slot_id" text,PRIMARY KEY (course_id));
CREATE TABLE teaches( "id" text,"course_id" text,"section_id" text,"semester" t